### Quantum Coin Flip Game with Cirq

#### What’s This All About?
Let’s dive into a quantum twist on a coin flip game! Alice sets up a qubit in the |+⟩ state—a perfect superposition—and hands it to Bob. Bob picks either the X-basis (|+⟩ or |-⟩) or Z-basis (|0⟩ or |1⟩) to measure it. If he gets |+⟩ or |0⟩, he wins; if it’s |-⟩ or |1⟩, Alice wins. We’ll run this 500 times to see who’s luckier. Then, Bob entangles his choice with another qubit—does that mess with the odds?

In [2]:
# Import Cirq and other essentials
import cirq
import numpy as np
import matplotlib.pyplot as plt

print("Cirq and friends imported—ready to roll!")

Cirq and friends imported—ready to roll!


#### Step 1: Define the Basic Game Circuit
First, we build the basic version without entanglement. Alice preps a single qubit, and Bob randomly picks a basis.

In [3]:
def basic_coin_flip_circuit():
    # Define a single qubit for Alice
    q0 = cirq.NamedQubit("Alice")
    
    # Create the circuit
    circuit = cirq.Circuit()
    
    # Alice applies H to put q0 in |+⟩
    circuit.append(cirq.H(q0))
    
    # Bob flips a coin for X or Z basis
    basis = np.random.choice(["X", "Z"])
    if basis == "X":
        # X-basis: H gate to measure in |+⟩/|-⟩
        circuit.append(cirq.H(q0))
    
    # Measure in computational (Z) basis; X-basis adjusts via H
    circuit.append(cirq.measure(q0, key="result"))
    
    return circuit, basis

# Quick peek at one circuit
circuit, basis = basic_coin_flip_circuit()
print("Basic circuit sample:")
print(circuit)
print(f"Bob chose {basis}-basis this time.")

Basic circuit sample:
Alice: ───H───M('result')───
Bob chose Z-basis this time.
